<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division
try: 
    import tensorflow as tf
    print(f'tensorflow: {tf.__version__}')
except ModuleNotFoundError as err:
    %pip install tensorflow
    try: 
        import tensorflow as tf
        print(f'\ntensorflow: {tf.__version__}')
    except Exception as err:
        print(err)
import glob, os, shutil
from os.path import *
import numpy as np
from google.colab import drive, files

contentPth = os.getcwd()
myDrivePth = '/content/drive/My Drive'
deletePth = '/content/sample_data'
if exists(deletePth):
    shutil.rmtree(deletePth)
    deletePth=None

drive.mount('/content/drive', force_remount=True)

os.chdir(myDrivePth)
from BashColors import C
os.chdir(contentPth)

mobilenet_v2=None

In [ ]:
physicalDevices = tf.config.experimental.list_physical_devices('GPU')
print(f'{len(physicalDevices)} gpu is available')
tf.config.experimental.set_memory_growth(physicalDevices[0], True)

In [ ]:
mobilenet_v2=tf.keras.applications.MobileNetV2(input_shape=None,
                                               alpha=1.0,
                                               include_top=True,
                                               weights='imagenet',
                                               input_tensor=None, 
                                               pooling=None,
                                               classes=1000,
                                               classifier_activation='softmax')

# mobilenet_v2.summary()
modelName = mobilenet_v2.name + '.h5'
print('saved: ', modelName)
mobilenet_v2.save(modelName)

In [ ]:
sourcePth = abspath('mobilenetv2_1.00_224.h5')
tfModelsPth = '/content/drive/My Drive/Tensorflow Models'
destinationPth = join(tfModelsPth, basename(sourcePth))
shutil.copy2(sourcePth, destinationPth)

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
from IPython.display import Image, display
from tensorflow.keras.applications import imagenet_utils
import numpy as np

imageList=None
imgPth = '/content/drive/My Drive/Test Images'
if exists(imgPth):
    os.chdir(imgPth)
    imageList=glob.glob('**')
    counter=0

for thisImage in sorted(imageList):
    counter+=1
    img=image.load_img(thisImage, target_size=(224,224))
    img_array=image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    result=tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)
    print(f'{counter}. {C.Blue}{thisImage}{C.ColorOff}')
    show=Image(filename=thisImage, width=112, height=113)
    display(show)
    thisPrediction = mobilenet_v2.predict(result)
    thisResult = imagenet_utils.decode_predictions(thisPrediction)
    print(f'{C.Purple}{thisResult[0]}{C.ColorOff}\n')
